## 📘 Simple RNN от посимвольной до пословной токенизации

<details> 
    <summary><em><strong> Рекуррентная нейронная сеть (RNN)</strong></em></summary>

## **1. Введение и мотивация**

### **1.1 Почему нужны рекуррентные сети**
- **Последовательные данные**: язык, временные ряды, аудио, ДНК‑последовательности.  
- **Зависимости во времени**: полносвязные сети считают входы независимыми; RNN хранят контекст в скрытом состоянии $h_t$.

### **1.2 История**   

- **1982 г. — Hopfield‑сеть.**  
  Показала, что нейронная сеть с симметричными весами может работать как энергетическая модель памяти‑ассоциаций. Работа Дж. Хопфилда стала первой демонстрацией тренируемых рекуррентных связей в нейро‑вычислениях.

- **1986 г. — алгоритм BPTT (Rumelhart & McClelland).**  
  Авторы обобщили классический back‑propagation на временно развёрнутые графы, что открыло путь к градиентному обучению длинных последовательностей. Книга *Parallel Distributed Processing* закрепила идею распределённых репрезентаций.

- **1990 г. — «Simple RNN» (Elman).**  
  Д. Элман показал, что рекуррентный «контекстный» слой способен захватывать грамматические зависимости в синтетическом языке. Так появилась базовая архитектура Elman‑net, ставшая учебным эталоном RNN.

- **1997 г. — LSTM (Hochreiter & Schmidhuber).**  
  Введение ячейки памяти и вентилирования решило проблему затухающих градиентов, позволив моделировать зависимости на сотни шагов назад. LSTM вскоре стал стандартом для речи и машинного перевода.

- **2014 г. — GRU (Cho и др.).**  
  Сократив число вентилей до двух, GRU предложил более лёгкую альтернативу LSTM при сопоставимой точности. Публикация совпала с бумом seq2seq‑моделей в переводе и диалоговых системах.

- **2020‑е — гибриды RNN + Attention (RWKV, S4, Mamba).**  
  Современные работы объединяют линейные рекуррентные операторы со слоем внимания, достигая масштабируемости трансформеров при памяти $O(1)$. Такие модели успешно конкурируют на задачах длинного контекста и стриминга.


## **2. Simple RNN (Ячейка Элмана): Как это работает?**

### **2.1 Интуиция**

Представьте, что вы читаете предложение слово за словом. Чтобы понять смысл текущего слова, вы используете не только само слово, но и контекст, накопленный из предыдущих слов. Simple RNN работает похожим образом:

*   На каждом временном шаге $t$ она принимает:
    1.  **Новый вход** $x_t$ (например, векторное представление слова).
    2.  **Состояние из предыдущего шага** $h_{t-1}$ (контекст, "память").
*   На основе этих двух входов она вычисляет:
    1.  **Новое состояние** $h_t$, которое будет передано на следующий шаг.
    2.  **Выход** $y_t$ (например, предсказание следующего слова или метка для текущего элемента).

### **2.2 Формализация и Обозначения**

Давайте опишем это математически. Сначала определимся с обозначениями и размерами тензоров (векторов/матриц):

| **Объект** | **Размерность**        | **Смысл**                                    |
| :--------- | :--------------------- | :------------------------------------------- |
| $x_t$      | $\mathbb{R}^{d_x}$     | Вектор входа в момент времени $t$            |
| $h_t$      | $\mathbb{R}^{d_h}$     | Вектор скрытого состояния в момент $t$       |
| $y_t$      | $\mathbb{R}^{d_y}$     | Вектор выхода модели в момент $t$            |
| $W_{xh}$   | $\mathbb{R}^{d_x \times d_h}$ | Матрица весов "вход → скрытое состояние"   |
| $W_{hh}$   | $\mathbb{R}^{d_h \times d_h}$ | Матрица весов "предыдущее состояние → текущее состояние" (рекуррентная связь) |
| $W_{hy}$   | $\mathbb{R}^{d_h \times d_y}$ | Матрица весов "скрытое состояние → выход" |
| $b_h$      | $\mathbb{R}^{d_h}$     | Вектор смещения для скрытого слоя            |
| $b_y$      | $\mathbb{R}^{d_y}$     | Вектор смещения для выходного слоя           |

> **Зачем следить за размерностями?** Это помогает избежать ошибок при матричных операциях и при написании кода (особенно с broadcast'ингом в библиотеках типа NumPy/PyTorch).

### **2.3 Динамика одного шага**

Теперь запишем формулы, описывающие переход от шага $t-1$ к шагу $t$:

$$
\boxed{%
\begin{aligned}
h_t &= \sigma_h\!\bigl(W_{xh}x_t + W_{hh}h_{t-1} + b_h\bigr), & h_0&=\mathbf0,\\[4pt]
y_t &= \sigma_y\!\bigl(W_{hy}h_t + b_y\bigr).
\end{aligned}}
$$

**Пояснения:**

1.  **Вычисление скрытого состояния $h_t$:**
    *   $W_{xh}x_t$: Влияние текущего входа $x_t$ на новое состояние.
    *   $W_{hh}h_{t-1}$: Влияние предыдущего состояния $h_{t-1}$ (памяти) на новое состояние. Это **ключевая рекуррентная связь**.
    *   $b_h$: Смещение (bias).
    *   $\sigma_h$: Функция активации скрытого слоя. Часто используют **tanh** или **сигмоиду**, так как они "сжимают" значения в ограниченный диапазон ([-1, 1] для tanh, [0, 1] для сигмоиды), что может помочь стабилизировать градиенты при обучении.
    *   $h_0 = \mathbf{0}$: Начинаем с нулевого вектора состояния перед обработкой первого элемента последовательности.

2.  **Вычисление выхода $y_t$:**
    *   $W_{hy}h_t$: Преобразование текущего скрытого состояния $h_t$ в выходное представление.
    *   $b_y$: Смещение выходного слоя.
    *   $\sigma_y$: Функция активации выходного слоя. Её выбор **зависит от задачи**:
        *   `softmax`: для задач классификации (например, предсказание следующего символа/слова из словаря).
        *   `sigmoid`: для бинарной классификации (например, анализ тональности: положительный/отрицательный).
        *   `id` (линейная активация, т.е. её отсутствие): для задач регрессии (предсказание числового значения).

![Image_01](https://raw.githubusercontent.com/Verbasik/Weekly-arXiv-ML-AI-Research-Review/refs/heads/develop/2025/week-17_&_18/assets/RNN/Image_01.webp)

```python
"""
Этот программный код представляет собой реализацию простой рекуррентной нейронной сети (RNN) для обработки последовательностей слов. 
Код включает в себя инициализацию параметров модели, функции для вычисления softmax, а также основной цикл RNN, который обрабатывает 
входную последовательность слов и выводит прогнозы для каждого слова в последовательности.

Функциональное назначение:
Код демонстрирует работу RNN на примере обработки текстовой последовательности. Он инициализирует веса и смещения, выполняет встроенные 
операции (one-hot encoding, embedding, вычисление скрытого состояния, softmax), и выводит топ-2 прогноза для каждого слова в последовательности.
"""

import numpy as np
import pandas as pd

def softmax(x: np.ndarray) -> np.ndarray:
    """
    Description:
    ---------------
        Вычисляет softmax для входного массива.

    Args:
    ---------------
        x: Входной массив, для которого нужно вычислить softmax.

    Returns:
    ---------------
        Массив с примененной функцией softmax.

    Raises:
    ---------------
        ValueError: Если входной массив пуст.

    Examples:
    ---------------
        >>> softmax(np.array([1, 2, 3]))
        array([0.09003057, 0.24472847, 0.66524096])
    """
    if x.size == 0:
        raise ValueError("Входной массив не может быть пустым")

    e = np.exp(x - np.max(x, axis=0, keepdims=True))
    return e / e.sum(axis=0, keepdims=True)

# ---------------- Параметры модели ----------------
vocab = ["the", "students", "opened", "their", "books", "laptops", "zoo"]
V = len(vocab)
d_e, d_h = 8, 16  # размеры embedding и скрытого состояния

# Словарь: слово → индекс
word2idx = {w: i for i, w in enumerate(vocab)}

# Инициализация весов
np.random.seed(0)
E = np.random.randn(d_e, V) * 0.1      # эмбеддинги
W_e = np.random.randn(d_h, d_e) * 0.1  # скрытое ← эмбеддинг
W_h = np.random.randn(d_h, d_h) * 0.1  # скрытое ← скрытое
b1 = np.zeros((d_h, 1))                # смещение для скрытого слоя
U = np.random.randn(V, d_h) * 0.1      # проекция скрытого → логиты
b2 = np.zeros((V, 1))                  # смещение для выходного слоя

# --------------- Визуализация матриц ----------------
df_E = pd.DataFrame(
    E, index=[f"e{i}" for i in range(d_e)], columns=vocab
)
df_We = pd.DataFrame(
    W_e, index=[f"h{i}" for i in range(d_h)], columns=[f"e{j}" for j in range(d_e)]
)
df_Wh = pd.DataFrame(
    W_h, index=[f"h{i}" for i in range(d_h)], columns=[f"h{j}" for j in range(d_h)]
)
df_U = pd.DataFrame(
    U, index=vocab, columns=[f"h{j}" for j in range(d_h)]
)

print("\nМатрица E (эмбеддинги):")
print(df_E)
print("\nМатрица W_e (скрытое ← эмбеддинг):")
print(df_We)
print("\nМатрица W_h (скрытое ← скрытое):")
print(df_Wh)
print("\nМатрица U (проекция на выход):")
print(df_U)

# --------------- Основной цикл RNN ----------------
sequence = ["the", "students", "opened", "their"]
h_prev = np.zeros((d_h, 1))

print("\nШаг  t    Слово      Топ‑2 (слово, вер‑ть)")
print("-" * 60)
for t, word in enumerate(sequence, 1):
    print(f"\n## Пошаговый разбор для t={t}, слово = '{word}'")

    # 1) One‑hot
    x = np.zeros((V, 1))
    x[word2idx[word], 0] = 1.0
    print("1) One‑hot вектор x:")
    print(x.T)

    # 2) Embedding
    e = E @ x
    print("\n2) Embedding e = E @ x:")
    print(e.T)

    # 3) Скрытое состояние
    h = np.tanh(W_h @ h_prev + W_e @ e + b1)
    print("\n3) Скрытое состояние h:")
    print(h.T)

    # 4) Логиты и softmax
    o = U @ h + b2
    y = softmax(o)
    print("\n4) Логиты o = U @ h + b2:")
    print(o.T)
    print("   Softmax y:")
    print(y.T)

    # Топ‑2 кандидата
    top2 = np.argsort(-y.flatten())[:2]
    probs = [(vocab[i], float(y[i])) for i in top2]
    print(f"\nТоп‑2 кандидата: {probs}")

    # Обновление скрытого состояния
    h_prev = h
```

### **Пояснения к схеме «Простая RNN‑языковая модель» (step by step)**

1. **Подача входа**  
   - На каждом шаге $t$ мы имеем слово в виде one‑hot вектора  
     $$x^{(t)} \in \mathbb{R}^{|V|}$$  
     где $|V|$ — размер словаря.
     
   - Пример: для словаря $\{\text{the}, \text{students}, \text{opened}, \dots\}$ слово «students» кодируется вектором, где на позиции «students» стоит 1, а в остальных — 0.

2. **Преобразование в embedding**  
   - Умножаем one‑hot $x^{(t)}$ на матрицу вложений  
     $$E \in \mathbb{R}^{d_e \times |V|}$$  
     чтобы получить плотный вектор  
     $$e^{(t)} = E \, x^{(t)} \in \mathbb{R}^{d_e}$$

3. **Обновление скрытого состояния**  
   - Рекуррентная формула:  

    $$
      h^{(t)} = \sigma\bigl(W_h \, h^{(t-1)} + W_e \, e^{(t)} + b_1\bigr)
    $$  
     
     где  
     - $h^{(t)} \in \mathbb{R}^{d_h}$ — скрытое состояние на шаге $t$,  
     - $W_h \in \mathbb{R}^{d_h \times d_h}$ — матрица перехода по скрытому состоянию,  
     - $W_e \in \mathbb{R}^{d_h \times d_e}$ — матрица для входного embedding,  
     - $b_1 \in \mathbb{R}^{d_h}$ — вектор смещений,  
     - $\sigma$ — нелинейность (обычно $\tanh$ или ReLU).

   Инициализация:  

     $$h^{(0)} = \mathbf{0}\quad(\text{или случайный вектор}).$$  
   - При расчёте выхода к $W_h\,h^{(t-1)} + W_e\,e^{(t)}$ прибавляется смещение $b_1$, а к $U\,h^{(t)}$ — смещение $b_2$, после чего по логитам вычисляется softmax.

4. **Вычисление выхода**  
   - Строим логиты для распределения по словарю:  
     $$
       o^{(t)} = U \, h^{(t)} + b_2,\qquad U\in\mathbb{R}^{|V|\times d_h},\;b_2\in\mathbb{R}^{|V|}.
     $$  
   - Применяем softmax, чтобы получить вероятностное распределение:  
     $$
       \hat y^{(t)} = \mathrm{softmax}\bigl(o^{(t)}\bigr)\in[0,1]^{|V|},\quad\sum_i \hat y^{(t)}_i = 1.
     $$  
   - Вектор $\hat y^{(t)}$ показывает, какое слово модель считает наиболее вероятным следующим на позиции $t+1$.

5. **Повторяем во времени**  
   - Матрицы весов ($W_{xh}, W_{hh}, W_{hy}$) и векторы смещений ($b_h, b_y$) **одни и те же на всех временных шагах $t$**. Сеть использует один и тот же набор параметров для обработки каждого элемента последовательности. Это делает RNN компактными по количеству параметров, независимо от длины последовательности $T$.

## **3. Обучение RNN: Backpropagation Through Time (BPTT)**

Мы определили, как RNN делает предсказания (прямой проход). Но как настроить её веса $W_{xh}, W_{hh}, W_{hy}, b_h, b_y$, чтобы предсказания были точными? Для этого нужен алгоритм обратного распространения ошибки, адаптированный для рекуррентной структуры — **Backpropagation Through Time (BPTT)**.

### **3.1 Идея: Разворачивание во времени**

Чтобы применить градиентный спуск, нам нужно вычислить градиенты функции потерь $L$ по всем параметрам модели. Сложность в том, что выход $y_t$ зависит от $h_t$, который зависит от $h_{t-1}$, который зависит от $h_{t-2}$, и так далее, вплоть до $h_0$. Кроме того, все $h_k$ (для $k < t$) зависят от одних и тех же весов $W_{hh}$ и $W_{xh}$.

Идея BPTT заключается в том, чтобы **мысленно "развернуть" RNN во времени** для последовательности длиной $T$. Представьте, что у вас есть $T$ копий одной и той же ячейки RNN, соединенных последовательно. Вход $x_t$ и предыдущее состояние $h_{t-1}$ подаются в $t$-ю копию, она выдает $h_t$ и $y_t$, и $h_t$ передается в $(t+1)$-ю копию.

![Image_02](https://raw.githubusercontent.com/Verbasik/Weekly-arXiv-ML-AI-Research-Review/refs/heads/develop/2025/week-17_&_18/assets/RNN/Image_02.png)

### **Пояснения к схеме «Простая RNN‑языковая модель» (step by step)**

Ниже показано, как на примере фразы «the students opened their» происходит прямой и обратный проходы (BPTT) в развёрнутой RNN-модели.

#### **1. Развёртывание по времени (Unrolling)**

- Каждый прямоугольник на схеме соответствует одному временному шагу $t=0,1,2,3$.  
- Входы: $x_0,x_1,x_2,x_3$ — one‑hot векторы слов «the», «students», «opened», «their».  
- Начальное скрытое состояние $h_{-1}$ инициализируется нулями.  
- Скрытые состояния $h_0\ldots h_3$ последовательно передаются по ребру $W_{hh}$.  
- На каждом шаге из $h_t$ через $W_{hy}$ вычисляется выход $\hat y_t$.

#### **2. Прямой проход (Forward pass)**

#### **Шаг 0 ($t=0$), слово «the»**

1. **One-hot представление**:  
   $ x_0 $ — единичный вектор, где единица находится в позиции слова «the».

2. **Embedding (векторное представление)**:  
   $ e_0 = E\,x_0 $,  
   где $ E $ — матрица эмбеддингов.

3. **Скрытое состояние (hidden state)**:  
   $$
   h_0 = \tanh\bigl(W_{xh} e_0 + W_{hh} h_{-1} + b_1\bigr),
   $$  
   где:
   - $ W_{xh}, W_{hh} $ — весовые матрицы,
   - $ b_1 $ — смещение,
   - $ h_{-1} $ — начальное скрытое состояние (обычно нулевой вектор).

4. **Выход модели и softmax**:  
   $$
   o_0 = W_{hy}h_0 + b_2,\quad \hat{y}_0 = \mathrm{softmax}(o_0),
   $$  
   где:
   - $ W_{hy} $ — матрица для преобразования скрытого состояния в логиты,
   - $ b_2 $ — смещение,
   - $ \hat{y}_0 $ — вероятностное распределение по словарю.

5. **Функция потерь (Loss)**:  
   Целевое слово — «students». Потеря вычисляется как:  
   $$
   L_0 = -\log\hat{y}_0[\text{students}].
   $$

#### **Подробнее о функции потерь**

Модель использует **кросс-энтропийную функцию потерь** для многоклассовой задачи. Рассмотрим её этапы:

1. **Логиты и вероятности**:  
   Модель выдаёт вектор логитов:  
   $$
   o_0 = W_{hy}h_0 + b_2,
   $$  
   который затем преобразуется в вероятности через softmax:  
   $$
   \hat{y}_0 = \mathrm{softmax}(o_0) \in [0, 1]^{|V|}, \quad \sum_i \hat{y}_0[i] = 1.
   $$

2. **Целевая метка**:  
   Целевая метка $ y^{(0)} $ — one-hot вектор, где единица стоит в позиции целевого слова «students»:  
   $$
   y^{(0)}_{\text{students}} = 1.
   $$

3. **Кросс-энтропия**:  
   Формула кросс-энтропии:  
   $$
   L_0 = -\sum_{i=1}^{|V|} y^{(0)}_i \log\hat{y}_0[i] = -\log\hat{y}_0[\text{students}].
   $$

4. **Интуиция**:  
   Чем меньше вероятность предсказанного слова $ \hat{y}_0[\text{students}] $, тем выше штраф (значение потери).

#### **Шаг 1 ($t=1$), слово «students»**
- Аналогично: $x_1$ → $e_1$ →  
  $$h_1 = \tanh(W_{xh}e_1 + W_{hh}h_0 + b_1).$$  
- Выход $\hat y_1 = \mathrm{softmax}(W_{hy}h_1+b_2)$,  
  целевое слово «opened», $L_1=-\log\hat y_1[opened]$.

#### **Шаг 2 ($t=2$), слово «opened»**
- $x_2$ → $e_2$ →  
  $$h_2 = \tanh(W_{xh}e_2 + W_{hh}h_1 + b_1).$$  
- $\hat y_2$, целевой «their», $L_2=-\log\hat y_2[their]$.

#### **Шаг 3 ($t=3$), слово «their»**
- $x_3$ → $e_3$ →  
  $$h_3 = \tanh(W_{xh}e_3 + W_{hh}h_2 + b_1).$$  
- $\hat y_3$, целевой «books», $L_3=-\log\hat y_3[books]$.

- **Суммарная потеря**:  
  $$L = L_0 + L_1 + L_2 + L_3.$$  

#### **3. Обратный проход (Backward pass — BPTT)**

- Градиенты от каждой $L_t$ (красные стрелки) прокатываются через:
  - выходной слой $W_{hy}$ к скрытым состояниям,
  - рекуррентные связи $W_{hh}$ к предыдущим $h_{t-1}$.
- На каждом шаге аккумулируются $
  \frac{\partial L}{\partial W_{xh}},
  \frac{\partial L}{\partial W_{hh}},
  \frac{\partial L}{\partial W_{hy}},
  \frac{\partial L}{\partial b_1},
  \frac{\partial L}{\partial b_2}$.
- В итоге веса обновляются с учётом вклада ошибок со всех временных шагов.

**Вывод:** BPTT разворачивает RNN во времени, вычисляет локальные потери на каждом шаге и распространяет ошибки сквозь все временные соединения, обеспечивая обучение с учётом контекстов предыдущих токенов.

Хотя мы создаем $T$ копий для вычислений, важно помнить: **веса $W_{xh}, W_{hh}, W_{hy}$ общие для всех этих копий**.

#### **3.2 Общая функция потерь**

Обычно общая потеря $L$ для всей последовательности — это сумма или среднее локальных потерь $\ell$ на каждом шаге:

$$
L \;=\;\sum_{t=1}^{T}\,\ell\bigl(y_t,\widehat y_t\bigr),
$$

где $y_t$ — предсказание модели на шаге $t$, а $\widehat y_t$ — истинное значение (цель) на шаге $t$. Функция $\ell$ может быть, например, кросс-энтропией для классификации или среднеквадратичной ошибкой (MSE) для регрессии.

#### **3.3 Вычисление градиентов (Пример для $W_{hh}$)**

Рассмотрим, как вычислить градиент общей потери $L$ по одному элементу $w$ из матрицы $W_{hh}$. Используя цепное правило, градиент $L$ по $w$ складывается из вкладов от каждого временного шага $t$:

$$
\frac{\partial L}{\partial w}\;=\; \sum_{t=1}^{T}\,\frac{\partial \ell(y_t, \widehat y_t)}{\partial w}
$$

Чтобы найти $\frac{\partial \ell(y_t, \widehat y_t)}{\partial w}$, нам нужно учесть, как $w$ влияет на $y_t$. Это влияние происходит через скрытое состояние $h_t$:

$$
\frac{\partial \ell(y_t, \widehat y_t)}{\partial w} = \frac{\partial \ell}{\partial y_t} \frac{\partial y_t}{\partial h_t} \frac{\partial h_t}{\partial w}
$$

Самая сложная часть — это $\frac{\partial h_t}{\partial w}$. Состояние $h_t$ зависит от $w$ напрямую (через член $W_{hh}h_{t-1}$ в формуле для $h_t$) и косвенно, через все предыдущие состояния $h_{t-1}, h_{t-2}, \dots, h_1$, так как они тоже зависят от $w$.

$$
\frac{\partial h_t}{\partial w} = \underbrace{\frac{\partial h_t}{\partial h_{t-1}}\frac{\partial h_{t-1}}{\partial w}}_{\text{через } h_{t-1}} + \underbrace{\frac{\partial h_t}{\partial w}}_{\text{прямое влияние}}
$$

Раскрывая эту рекурсию дальше, мы увидим, что градиент включает в себя **сумму путей** разной длины из прошлого в настоящее. Каждый такой путь включает произведения Якобианов $\frac{\partial h_k}{\partial h_{k-1}}$.

$$
\frac{\partial h_k}{\partial h_{k-1}} = \frac{\partial}{\partial h_{k-1}} \sigma_h(W_{xh}x_k + W_{hh}h_{k-1} + b_h) = \operatorname{diag}\!\bigl[\sigma_h'(a_k)\bigr]\,W_{hh}
$$
где $a_k = W_{xh}x_k + W_{hh}h_{k-1} + b_h$ — аргумент функции активации $\sigma_h$ на шаге $k$. Обозначим этот Якобиан как $J_k$.

Тогда вклад в градиент от пути длиной $k$ (от $h_{t-k}$ к $h_t$) будет включать произведение $k$ таких Якобианов: $J_t J_{t-1} \dots J_{t-k+1}$.

#### **3.4 Проблемы: Затухание и Взрыв Градиентов**

Именно эти **длинные произведения Якобианов** $J_k = \operatorname{diag}[\sigma_h'(a_k)] W_{hh}$ являются источником проблем при обучении RNN:

1.  **Затухание градиента (Vanishing Gradient):** Если собственные значения матрицы $W_{hh}$ (или нормы Якобианов $J_k$) по модулю **меньше 1**, то при умножении многих таких матриц результат будет стремиться к нулю экспоненциально быстро с ростом $k$. Это означает, что градиенты от далеких прошлых шагов ($t-k$ для больших $k$) почти не доходят до параметров $W_{hh}$, и сеть не может научиться **долговременным зависимостям**. Simple RNN особенно подвержены этой проблеме.
2.  **Взрыв градиента (Exploding Gradient):** Если собственные значения $W_{hh}$ (или нормы $J_k$) по модулю **больше 1**, то произведение Якобианов будет расти экспоненциально. Это приводит к огромным значениям градиентов, что делает шаги градиентного спуска нестабильными и может привести к расхождению обучения (NaN/Inf в потерях или весах).

#### **3.5 Классические Решения**

*   **Gradient Clipping:** Искусственное ограничение нормы градиента. Если $\|\nabla\theta\| > \tau$ (некоторый порог), то градиент масштабируется: $\nabla\theta \leftarrow \frac{\tau}{\|\nabla\theta\|} \nabla\theta$. Это помогает бороться со *взрывом*, но не с *затуханием*.
*   **Правильная инициализация весов:** Например, ортогональная инициализация для $W_{hh}$ может помочь держать собственные значения близкими к 1.
*   **Использование более сложных ячеек:** **LSTM (Long Short-Term Memory)** и **GRU (Gated Recurrent Unit)** были разработаны специально для борьбы с затуханием градиентов. Они вводят "вентили" (gates), которые контролируют поток информации и градиентов через ячейку, позволяя сохранять информацию на долгие периоды.
*   **Функции активации:** Использование ReLU может усугубить взрыв градиента, но менее подвержено затуханию, чем сигмоида/tanh (если активация не нулевая). Однако в рекуррентной части часто предпочитают tanh.


## **4. BPTT на Практике: Квази-код и PyTorch**

Современные фреймворки глубокого обучения (PyTorch, TensorFlow/Keras) реализуют BPTT автоматически. Вам нужно лишь определить архитектуру RNN и запустить обратный проход (`loss.backward()` в PyTorch).

Вот как выглядит типичный цикл обучения с использованием BPTT в PyTorch (с использованием `tanh` как $\sigma_h$ и линейной $\sigma_y$):

```python
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# --- Гиперпараметры и данные (примерные) ---
T = 10      # Длина последовательности
batch_size = 32
d_x = 20    # Размерность входа
d_h = 50    # Размерность скрытого состояния
d_y = 5     # Размерность выхода

# --- Модель (определяем параметры) ---
W_xh = torch.randn(d_x, d_h, requires_grad=True)
W_hh = torch.randn(d_h, d_h, requires_grad=True)
W_hy = torch.randn(d_h, d_y, requires_grad=True)
b_h  = torch.zeros(d_h, requires_grad=True)
b_y  = torch.zeros(d_y, requires_grad=True)
params = [W_xh, W_hh, W_hy, b_h, b_y]

# --- Пример данных ---
x_sequence = torch.randn(T, batch_size, d_x) # [Время, Батч, Признаки]
y_true_sequence = torch.randn(T, batch_size, d_y)

# --- Оптимизатор ---
optimizer = optim.Adam(params, lr=0.001)

# --- Цикл обучения (одна итерация) ---
optimizer.zero_grad()

# == Forward pass (разворачивание цикла вручную для ясности) ==
h_t = torch.zeros(batch_size, d_h) # Начальное скрытое состояние h_0
outputs = []
for t in range(T):
    # Формула Simple RNN
    h_t = torch.tanh(x_sequence[t] @ W_xh + h_t @ W_hh + b_h)
    y_t = h_t @ W_hy + b_y # Линейный выходной слой
    outputs.append(y_t)

# Собираем выходы в один тензор [T, Batch, d_y]
y_pred_sequence = torch.stack(outputs)

# == Вычисление потерь ==
# Пример: MSE на каждом шаге, затем усредняем по времени и батчу
loss = F.mse_loss(y_pred_sequence, y_true_sequence)

# == Backward pass (BPTT) ==
loss.backward() # PyTorch автоматически вычисляет градиенты ∂L/∂params через BPTT

# == Опционально: Gradient Clipping ==
torch.nn.utils.clip_grad_norm_(params, max_norm=1.0) # Ограничиваем норму градиента

# == Шаг оптимизатора ==
optimizer.step()

print(f"Loss: {loss.item()}")
# print(f"Gradient norm for W_hh: {W_hh.grad.norm().item()}") # Можно посмотреть на норму градиента
```

**Ключевые моменты:**

*   PyTorch строит динамический вычислительный граф во время forward pass.
*   Когда вызывается `loss.backward()`, PyTorch проходит по этому графу в обратном порядке, применяя цепное правило (реализуя BPTT) для вычисления градиентов всех параметров (`requires_grad=True`), от которых зависит `loss`.
*   `torch.nn.utils.clip_grad_norm_` — стандартная практика для предотвращения взрыва градиентов.

## **5. Проблема долговременных зависимостей**

Одна из привлекательных идей RNN состоит в том, что они потенциально умеют связывать предыдущую информацию с текущей задачей, так, например, знания о предыдущем кадре видео могут помочь в понимании текущего кадра. Если бы RNN обладали такой способностью, они были бы чрезвычайно полезны. Но действительно ли RNN предоставляют нам такую возможность? Это зависит от некоторых обстоятельств.

Иногда для выполнения текущей задачи нам необходима только недавняя информация. Рассмотрим, например, языковую модель, пытающуюся предсказать следующее слово на основании предыдущих. Если мы хотим предсказать последнее слово в предложении “облака плывут по небу”, нам не нужен более широкий контекст; в этом случае довольно очевидно, что последним словом будет “небу”. В этом случае, когда дистанция между актуальной информацией и местом, где она понадобилась, невелика, RNN могут обучиться использованию информации из прошлого.

Но бывают случаи, когда нам необходимо больше контекста. Допустим, мы хотим предсказать последнее слово в тексте “Я вырос во Франции… Я бегло говорю по-французски”. Ближайший контекст предполагает, что последним словом будет называние языка, но чтобы установить, какого именно языка, нам нужен контекст Франции из более отдаленного прошлого. Таким образом, разрыв между актуальной информацией и точкой ее применения может стать очень большим.

К сожалению, по мере роста этого расстояния, RNN теряют способность связывать информацию.

![Image_03](https://raw.githubusercontent.com/Verbasik/Weekly-arXiv-ML-AI-Research-Review/refs/heads/develop/2025/week-17_&_18/assets/RNN/Image_03.png)

К счастью, LSTM не знает таких проблем!

## **Заключение**

Рекуррентные нейронные сети, обладая компактной памятью и естественной каузальностью, продолжают оставаться незаменимыми для потоковых задач и ситуаций с ограниченными ресурсами. Глубокое понимание их математической базы, инженерных приёмов и современных вариаций даёт исследователю инструмент, который гармонично дополняет «семейство» Transformer‑подобных моделей.

</details>

### Подключим необходимые пакеты

In [ ]:
"""
Реализация моделей RNN с посимвольной и пословной токенизацией для генерации текста.

Данный модуль обеспечивает создание, обучение и применение рекуррентных нейронных сетей
для генерации текста на русском языке. Реализованы два подхода: символьная и словесная токенизация.
Основной целью является демонстрация базовых принципов работы рекуррентных моделей
и сравнение эффективности различных методов токенизации.
"""

# Библиотеки для работы с данными и базами данных
import sqlite3
from collections import Counter
from typing import List, Dict, Any, Optional, Union, Tuple

import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split

# Библиотеки для обработки текста
import nltk
from nltk.tokenize import sent_tokenize

# Библиотеки глубокого обучения
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Библиотеки для визуализации и анализа
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Утилиты
from tqdm import tqdm

# Настройка визуализации
seaborn.set(palette='summer')

# Определение устройства для вычислений
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

### Load Data

В качестве датасета испльзуюся старницы Wiki на русском языке.
wikibooks-dataset

In [3]:
conn = sqlite3.connect('/kaggle/input/wikibooks-dataset/wikibooks.sqlite')

df = pd.read_sql_query("SELECT * FROM ru LIMIT 3300", conn)

### Preprocessing data

Разобьём страницы на тексты длиной 256 символов

In [ ]:
# Извлечение предложений из текстов
sentences = []

for sentence in tqdm(df['body_text']):
    sentences.extend(
        [x.lower() for x in sent_tokenize(sentence, language='russian') if len(x) < 256]
        )
    
print("Количество предложений", len(sentences))

100%|██████████| 3300/3300 [00:10<00:00, 319.92it/s]

Количество предложений 120873


### 📚 Токенизация на уровне символов: построение словаря

<details> 
    <summary><em><strong>построение словаря 👈</strong></em></summary>

Теперь мы приступим к важному этапу подготовки данных — **токенизации на уровне символов**. Этот шаг лежит в основе создания языковой модели, поскольку позволяет преобразовать текст в числовую форму, пригодную для обработки нейронной сетью.

#### 🔍 Что происходит в коде?

Давайте разберёмся по шагам, что делает наш блок кода:

1. **Определение `stop_chars`**  
   Мы задаём список знаков препинания и специальных символов, которые хотим исключить из словаря. Это помогает уменьшить его размер и сосредоточиться на более значимых символах.

2. **Подсчёт частоты символов**  
   Используем `Counter` из библиотеки `collections`, чтобы определить, как часто встречается каждый символ в корпусе текста.

3. **Фильтрация и перебор**  
   Проходим по всем предложениям и символам, игнорируя те, что находятся в списке `stop_chars`.

4. **Инициализация словаря (`vocab`)**  
   Создаём начальный словарь, включающий четыре **специальных токена**, о которых поговорим чуть ниже.

5. **Порог отбора (`counter_threshold = 500`)**  
   Устанавливаем порог, согласно которому в словарь войдут только те символы, которые встречаются чаще 500 раз.

6. **Формирование финального словаря**  
   Добавляем в `vocab` только те символы, которые соответствуют установленному порогу.

#### ⚙️ Специальные токены и их роль

Словарь начинается с четырёх **специальных токенов**, которые играют ключевую роль в обучении и генерации текста:

| Токен    | Название                    | Описание |
|----------|-----------------------------|----------|
| `<unk>`  | *Unknown*                   | Используется для неизвестных или редких символов, которых нет в словаре. |
| `<bos>`  | *Beginning of Sequence*     | Обозначает начало последовательности (например, предложения). Помогает модели понять контекст начала текста. |
| `<eos>`  | *End of Sequence*           | Сигнализирует о завершении последовательности. Важно для обучения модели корректно заканчивать генерацию. |
| `<pad>`  | *Padding*                   | Токен заполнения, используется для выравнивания длины последовательностей в пакете. |

Эти токены дают модели возможность работать с различными длинами предложений и адекватно интерпретировать начало и конец текста.

#### 📉 Почему мы используем пороговое значение?

Установка порога (`counter_threshold = 500`) — это стратегическое решение, которое:

- ✅ **Уменьшает размер словаря**, делая модель более компактной и эффективной
- ✅ **Фокусирует обучение на наиболее часто встречающихся символах**
- ✅ **Снижает риск переобучения** на редкие или случайные комбинации
- ✅ **Ускоряет обучение** и уменьшает требования к памяти

Благодаря этому подходу мы сохраняем важную информацию, игнорируя шум и малозначимые детали.

#### 🧱 Итог

Этот этап — основа всей последующей работы модели. Посимвольная токенизация даёт модели возможность учиться на тонких закономерностях текста, таких как пунктуация, регистр, морфология и даже стиль. С правильно построенным словарём и чёткой системой токенов мы готовы двигаться дальше — к созданию самой языковой модели.

</details> 

In [ ]:
# Обработка и токенизация на уровне символов
stop_chars = [
    "\t", ",", ".", "!", "@", "'", '"', ";", "\n", "(", ")",
    "[", "]", "{", "}", "?", ":", "-", "_", "+", "=", "^", "*", 
    "&", "`", "~"
]

chars = Counter()

for sentence in tqdm(sentences):
    for char in sentence:
        if char in stop_chars:
            continue
        chars[char] += 1
        
vocab = set(['<unk>', '<bos>', '<eos>', '<pad>'])
counter_threshold = 500

for char, cnt in chars.items():
    if cnt > counter_threshold:
        vocab.add(char)
        
print("Размер словаря:", len(vocab))

100%|██████████| 120873/120873 [00:08<00:00, 13696.57it/s]

Размер словаря: 91


In [6]:
char2ind = {char: i for i, char in enumerate(vocab)}
ind2char = {i: char for char, i in char2ind.items()}

### 🧪 Подготовка данных для обучения RNN: от текста к тензорам

<details> 
    <summary><em><strong>от текста к тензорам 👈</strong></em></summary>

Теперь, когда у нас есть **словарь символов**, мы можем перейти к следующему этапу — подготовке данных для обучения рекуррентной нейронной сети (RNN). Этот этап преобразует "сырой" текст в последовательности чисел, а затем — в тензоры, с которыми работает модель.

#### 📦 1. Создание специализированного датасета (`CharDataset`)

```python
class CharDataset:
    # ... код класса ...
```

Для удобства работы с данными мы создаём собственный класс `CharDataset`, который:

- ✅ Хранит список предложений и предоставляет к ним доступ по индексу
- ✅ Токенизирует текст "на лету", используя наш словарь
- ✅ Добавляет специальные токены `<bos>` в начало и `<eos>` в конец каждого предложения
- ✅ Заменяет неизвестные символы на `<unk>`, чтобы избежать ошибок

Когда вы вызываете `dataset[i]`, метод `__getitem__` возвращает **последовательность числовых индексов**, соответствующих символам предложения. Это первый шаг к тому, чтобы данные стали "понятны" нейросети.

#### 🔄 2. Упаковка данных в батчи с выравниванием (`collate_fn_with_padding`)

```python
def collate_fn_with_padding(input_batch, pad_id=char2ind['<pad>']):
    # ... код функции ...
```

Поскольку предложения имеют разную длину, нам нужно их **выравнивать** внутри батча. Для этого используется функция `collate_fn_with_padding`, которая:

- 🔁 Определяет максимальную длину последовательности в текущем батче
- ⬛ Добавляет паддинг-токены `<pad>` ко всем коротким последовательностям
- 🧮 Преобразует списки в тензоры PyTorch
- 🎯 Формирует пары `(input_ids, target_ids)` для обучения языковой модели

##### Пример вход-выход:
Если исходное предложение:  
`<bos>привет<eos>`

Тогда:
- `input_ids`: `[<bos>, п, р, и, в, е]`
- `target_ids`: `[п, р, и, в, е, т]`

Модель учится предсказывать **следующий символ** на основе всех предыдущих. Именно так реализуется задача языкового моделирования на уровне символов.

#### 🚀 3. Создание загрузчиков данных

```python
# Разделение данных на обучающую и тестовую выборки
train_sentences, eval_sentences = train_test_split(sentences, test_size=0.2)

# Создание датасетов и загрузчиков данных
train_dataset = CharDataset(train_sentences)
eval_dataset = CharDataset(eval_sentences)

train_dataloader = DataLoader(
    train_dataset, 
    collate_fn=collate_fn_with_padding, 
    batch_size=256
)

eval_dataloader = DataLoader(
    eval_dataset, 
    collate_fn=collate_fn_with_padding, 
    batch_size=256
)
```

На этом этапе мы:

1. 📊 Делим данные на обучающую (80%) и валидационную (20%) выборки
2. 📚 Создаем экземпляры `CharDataset` для каждой выборки
3. 🚣‍♂️ Инициализируем `DataLoader`'ы, которые:
   - 📦 Формируют батчи размером 256 предложений
   - 🔀 Перемешивают данные между эпохами
   - 📐 Применяют функцию `collate_fn_with_padding` для обработки батча
   - 📈 Автоматически преобразуют данные в тензоры PyTorch

Эти загрузчики обеспечивают **эффективную подачу данных** в модель во время обучения и оценки.

#### 🧠 Важно запомнить

Весь процесс подготовки данных можно представить как конвейер:

**Текст → Символы → Токены → Индексы → Тензоры**

Эта цепочка позволяет модели эффективно учиться на текстовых данных любого объёма и структуры. Особенно важно то, что вся логика токенизации и выравнивания скрыта внутри датасета и функции `collate_fn`, что делает обучение чистым, модульным и масштабируемым.

</details> 

Определим класс CharDataset для последующей загрузки его в dataloader

In [ ]:
class CharDataset:
    """
    Description:
    ---------------
        Датасет для работы с посимвольной токенизацией текста.

    Args:
    ---------------
        sentences: Список предложений для обработки

    Examples:
    ---------------
        >>> dataset = CharDataset(sentences)
        >>> sample = dataset[0]
        >>> len(sample)
        # Длина токенизированного предложения
    """
    def __init__(self, sentences: List[str]) -> None:
        self.data = sentences
        self.unk_id = char2ind['<unk>']
        self.bos_id = char2ind['<bos>']
        self.eos_id = char2ind['<eos>']
        self.pad_id = char2ind['<pad>']

    def __getitem__(self, idx: int) -> List[int]:
        """
        Description:
        ---------------
            Преобразует предложение в последовательность индексов символов.

        Args:
        ---------------
            idx: Индекс предложения в датасете

        Returns:
        ---------------
            List[int]: Токенизированное предложение как список индексов
        """
        tokenized_sentence = [self.bos_id]
        tokenized_sentence += [
            char2ind.get(char, self.unk_id) for char in self.data[idx]
        ]
        tokenized_sentence += [self.eos_id]

        return tokenized_sentence

    def __len__(self) -> int:
        """
        Description:
        ---------------
            Возвращает размер датасета.

        Returns:
        ---------------
            int: Количество предложений в датасете
        """
        return len(self.data)

Определим функцию дополнения предложений до max_seq_len

In [ ]:
def collate_fn_with_padding(
    input_batch: List[List[int]], 
    pad_id: int = char2ind['<pad>']
) -> Dict[str, torch.Tensor]:
    """
    Description:
    ---------------
        Функция для преобразования батча данных: добавляет паддинг и 
        создает тензоры ввода и целевых значений.

    Args:
    ---------------
        input_batch: Пакет токенизированных предложений
        pad_id: Идентификатор токена для заполнения (паддинга)

    Returns:
    ---------------
        Dict[str, torch.Tensor]: Словарь с тензорами входных и целевых id
    """
    seq_lens = [len(x) for x in input_batch]
    max_seq_len = max(seq_lens)

    new_batch = []
    for sequence in input_batch:
        for _ in range(max_seq_len - len(sequence)):
            sequence.append(pad_id)
        new_batch.append(sequence)

    sequences = torch.LongTensor(new_batch).to(device)

    new_batch = {
        'input_ids': sequences[:, :-1],
        'target_ids': sequences[:, 1:]
    }

    return new_batch

Создадим dataloader для train и eval датасетов

In [ ]:
# Разделение данных на обучающую и тестовую выборки
train_sentences, eval_sentences = train_test_split(
    sentences, test_size=0.2
)

# Создание датасетов и загрузчиков данных
train_dataset = CharDataset(train_sentences)
eval_dataset = CharDataset(eval_sentences)

train_dataloader = DataLoader(
    train_dataset, 
    collate_fn=collate_fn_with_padding, 
    batch_size=256
)

eval_dataloader = DataLoader(
    eval_dataset, 
    collate_fn=collate_fn_with_padding, 
    batch_size=256
)

### 🧠 Обучение рекуррентной нейронной сети: от теории к практике

<details> 
    <summary><em><strong>от теории к практике 👈</strong></em></summary>

Теперь, когда данные подготовлены, мы можем приступить к обучению модели. Этот этап — сердце всего процесса создания языковой модели: модель будет учиться предсказывать следующий символ на основе предыдущих.

#### 🎯 Зачем мы это делаем?

Цель обучения — научить модель **понимать** закономерности в последовательностях символов. Модель должна "выучить" не только грамматику и словарный запас, но и стилистику, частотные паттерны и даже логику построения предложений.

#### 🚦 1. Функция `fit_epoch`: обучение на одной эпохе

```python
def fit_epoch(model, train_loader, criterion, optimizer):
    # ... код ...
```

Эта функция реализует **один полный проход** через все данные обучающей выборки (эпоха). Вот основные шаги:

1. **`model.train()`** — активируем режим обучения  
   - Включаются такие механизмы, как Dropout или BatchNorm
   - Включается вычисление градиентов

2. **Обработка батч за батчем**  
   - Данные разбиваются на порции для эффективного использования памяти

3. **Для каждого батча:**
   - ❌ `optimizer.zero_grad()` — очищаем старые градиенты
   - 🔁 Прямое распространение (`model(input_ids)`) — получаем предсказания
   - 📉 Вычисляем loss — сравниваем предсказания с реальными символами
   - ⬅️ `loss.backward()` — вычисляем градиенты
   - ✅ `optimizer.step()` — обновляем веса модели

4. **Отслеживание метрик**
   - Сохраняем значение loss
   - Считаем перплексию: `torch.exp(loss)`  
     Это стандартная метрика качества языковых моделей

> 💡 **Перплексия (Perplexity)** — это мера "неожиданности" текста для модели:
- Перплексия = 1 → модель идеально предсказывает следующий символ
- Перплексия = 10 → модель в среднем рассматривает 10 возможных следующих символов
- Чем ниже — тем лучше!

#### 🔍 2. Функция `eval_epoch`: оценка на валидационной выборке

```python
def eval_epoch(model, val_loader, criterion):
    # ... код ...
```

Функция `eval_epoch` проверяет, **насколько хорошо модель обобщает знания** на новых данных, которые она не видела при обучении.

Основные особенности:
- **`model.eval()`** — переходим в режим оценки:
  - Отключается Dropout
  - Не вычисляются градиенты
- **Контекст `with torch.no_grad()`** — экономим память и повышаем производительность
- **Вычисляем те же метрики**, что и при обучении — чтобы можно было сравнить результаты

##### Почему важна валидация?
- Позволяет **обнаружить переобучение** (когда модель "зубрит", а не "понимает")
- Помогает выбрать **момент остановки** обучения
- Указывает на **реальное качество модели**

#### 🏋️‍♂️ 3. Функция `train`: полный цикл обучения

```python
def train(...):
    # ... код ...
```

Это "дирижёр" всего процесса. Он координирует работу `fit_epoch` и `eval_epoch`, управляет оптимизатором и контролирует сохранение лучшей модели.

Что происходит внутри:

##### 🔧 Настройка инструментов обучения
- Инициализация **оптимизатора** (`Adam`) — алгоритм обновления весов
- Инициализация **функции потерь** (`CrossEntropyLoss`) — способ сравнения предсказаний и целевых значений
- Указание `ignore_index=char2ind['<pad>']` — игнорируем лишние паддинги при обучении

##### 🛑 Early Stopping
- Отслеживаем **лучшую модель** по минимальной перплексии
- Сохраняем **веса лучшей модели**
- Прерываем обучение, если улучшений нет долгое время

##### 📈 Логирование и визуализация
- Сохраняем историю метрик
- Используем `tqdm` для красивого прогресс-бара
- Выводим понятную таблицу результатов после каждой эпохи

#### 🔄 Как всё работает вместе?

Процесс обучения строится как цикл:

```
ПОКА НЕ ДОСТИГНУТО УСЛОВИЕ ОСТАНОВКИ:
    1. Пройти по всем данным (одна эпоха)
    2. Обновить веса модели
    3. Протестировать на валидации
    4. Если результат лучше — сохранить модель
```

По завершении обучения загружается **лучшая версия модели**, найденная за весь процесс.

#### 🧮 Итак, что мы измеряем?

| Метрика       | Что показывает                              | Интерпретация                        |
|---------------|----------------------------------------------|---------------------------------------|
| Loss          | Средняя величина ошибки                      | Меньше — лучше                         |
| Perplexity    | Экспонента от loss                           | Ближе к 1 — лучше                      |

Модель стремится минимизировать loss и, соответственно, перплексию — это и есть **обучение**.

#### 📦 Подводя итог

Весь процесс обучения можно представить как:

**Данные → Прогноз → Ошибка → Обновление → Лучший прогноз**

Модель учится "предугадывать" следующий символ, опираясь на предыдущие. Так, шаг за шагом, она осваивает язык и становится способной не только воспроизводить, но и **генерировать** текст.

</details> 

### Train loop

In [ ]:
def fit_epoch(
    model: nn.Module, 
    train_loader: DataLoader, 
    criterion: nn.Module, 
    optimizer: torch.optim.Optimizer, 
    sheduler: Optional[torch.optim.lr_scheduler._LRScheduler] = None
) -> Tuple[float, float]:
    """
    Description:
    ---------------
        Выполняет одну эпоху обучения модели.

    Args:
    ---------------
        model: Модель для обучения
        train_loader: Загрузчик обучающих данных
        criterion: Функция потерь
        optimizer: Оптимизатор
        sheduler: Планировщик скорости обучения (опционально)

    Returns:
    ---------------
        Tuple[float, float]: Перплексия и значение функции потерь
    """
    model.train()
    running_loss = 0.0
    running_corrects = 0
    processed_data = 0
    losses = []
    perplexity = []
    
    for batch in train_loader:
        optimizer.zero_grad()

        # Распространение прямое и обратное
        logits = model(batch['input_ids']).flatten(start_dim=0, end_dim=1)
        loss = criterion(
            logits, batch['target_ids'].flatten()
        )
        loss.backward()
        optimizer.step()
        
        # Сохранение метрик
        perplexity.append(torch.exp(loss).item())
        losses.append(loss.item())
        
    # Расчет средних значений метрик
    perplexity_avg = sum(perplexity) / len(perplexity)
    losses_avg = sum(losses) / len(losses)    
    
    return perplexity_avg, losses_avg

In [ ]:
def eval_epoch(
    model: nn.Module, 
    val_loader: DataLoader, 
    criterion: nn.Module
) -> Tuple[float, float]:
    """
    Description:
    ---------------
        Оценивает модель на валидационном наборе данных.

    Args:
    ---------------
        model: Модель для оценки
        val_loader: Загрузчик валидационных данных
        criterion: Функция потерь

    Returns:
    ---------------
        Tuple[float, float]: Перплексия и значение функции потерь
    """
    model.eval()
    perplexity = []
    losses = []
    
    with torch.no_grad():
        for batch in val_loader:
            logits = model(batch['input_ids']).flatten(start_dim=0, end_dim=1)
            loss = criterion(
                logits,
                batch['target_ids'].flatten()
            )
            perplexity.append(torch.exp(loss).item())
            losses.append(loss.item())

    # Расчет средних значений метрик
    perplexity_avg = sum(perplexity) / len(perplexity)
    losses_avg = sum(losses) / len(losses)
    
    return perplexity_avg, losses_avg

In [ ]:
def train(
    train_dataloader: DataLoader, 
    eval_dataloader: DataLoader, 
    model: nn.Module, 
    epochs: int, 
    ignore_index: int = char2ind['<pad>'],
    optimizer: Optional[torch.optim.Optimizer] = None, 
    criterion: Optional[nn.Module] = None, 
    sheduler: Optional[torch.optim.lr_scheduler._LRScheduler] = None
) -> Tuple[nn.Module, List[Tuple[float, float, float, float]]]:
    """
    Description:
    ---------------
        Обучает модель на заданном количестве эпох.

    Args:
    ---------------
        train_dataloader: Загрузчик обучающих данных
        eval_dataloader: Загрузчик валидационных данных
        model: Модель для обучения
        epochs: Количество эпох обучения
        ignore_index: Индекс токена, который не учитывается в функции потерь
        optimizer: Оптимизатор (по умолчанию Adam)
        criterion: Функция потерь (по умолчанию CrossEntropyLoss)
        sheduler: Планировщик скорости обучения (опционально)

    Returns:
    ---------------
        Tuple[nn.Module, List[Tuple[float, float, float, float]]]: 
            Обученная модель и история обучения
    """
    # Инициализация оптимизатора и функции потерь, если не указаны
    if optimizer is None:
      optimizer = torch.optim.Adam(model.parameters())

    if criterion is None:
      criterion = nn.CrossEntropyLoss(ignore_index=ignore_index)

    # Сохранение лучших весов модели
    best_model_wts = model.state_dict()
    best_perplexity = 10e10

    # История обучения
    history = []
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} \
    val_loss {v_loss:0.4f} train_perplexirty {t_acc:0.4f} val_perplexirty {v_acc:0.4f}"

    with tqdm(desc="epoch", total=epochs) as pbar_outer:
        for epoch in range(epochs):
            # Обучение на одной эпохе
            train_perplexirty, train_loss = fit_epoch(
                model, train_dataloader, criterion, optimizer
            )
            
            # Валидация модели
            val_perplexirty, val_loss = eval_epoch(
                model, eval_dataloader, criterion
            )
            
            # Сохранение метрик
            history.append((train_loss, train_perplexirty, val_loss, val_perplexirty))
            
            # Сохранение лучшей модели
            if val_perplexirty < best_perplexity:
                best_perplexity = val_perplexirty
                best_model_wts = model.state_dict()

            # Обновление прогресс-бара и вывод результатов
            pbar_outer.update(1)
            tqdm.write(log_template.format(
                ep=epoch+1, 
                t_loss=train_loss,
                v_loss=val_loss, 
                t_acc=train_perplexirty, 
                v_acc=val_perplexirty
            ))

    print('Best val perplexirty: {:4f}'.format(best_perplexity))
    
    # Загрузка лучших весов
    model.load_state_dict(best_model_wts)

    return model, history

### 🧠 Архитектура языковой модели: Simple RNN на базе GRU

<details> 
    <summary><em><strong>Simple RNN 👈</strong></em></summary>

Теперь давайте посмотрим на **архитектуру нашей языковой модели**. Мы будем использовать рекуррентную нейронную сеть (RNN), а точнее её современную реализацию — **GRU (Gated Recurrent Unit)**. Это позволяет модели эффективно работать с последовательностями и "запоминать" контекст.

#### 🔨 Основной класс модели `CharLM`

```python
class CharLM(nn.Module):
    # ... код класса ...
```

Это наша модель для **посимвольного языкового моделирования**. Она состоит из нескольких ключевых слоёв:

---

#### 1. Слой эмбеддингов (`self.embedding`)

```python
self.embedding = nn.Embedding(vocab_size, hidden_dim)
```

- **Что делает**: преобразует индексы символов в плотные векторы (эмбеддинги)
- **Зачем нужно**: позволяет модели работать с символами как с числами в многомерном пространстве, где семантически близкие символы будут расположены рядом
- **Вход/Выход**:
  - Вход: `[batch_size, seq_len]`
  - Выход: `[batch_size, seq_len, hidden_dim]`

---

#### 2. Рекуррентный слой GRU (`self.rnn`)

```python
self.rnn = nn.GRU(hidden_dim, hidden_dim, batch_first=True)
```

- **Что делает**: обрабатывает последовательности, сохраняя информацию о предыдущих символах
- **Как работает**:
  - Использует **update gate** и **reset gate**, чтобы контролировать поток информации
  - Может запоминать зависимости на длинных последовательностях
- **Преимущества над обычным RNN**:
  - Лучше справляется с проблемой исчезающих градиентов
  - Более устойчив к долгосрочным зависимостям
- **Параметр `batch_first=True`**: говорит PyTorch, что первый размер — это размер батча

---

#### 3. Дополнительные слои обработки

```python
self.linear = nn.Linear(hidden_dim, hidden_dim)
self.non_lin = nn.Tanh()
self.dropout = nn.Dropout(p=0.1)
```

- **Линейный слой (`linear`)**: тонкая настройка скрытых представлений
- **Нелинейность (`Tanh`)**: добавляет нелинейность в модель, увеличивая её выразительность
- **Dropout (`p=0.1`)**: уменьшает переобучение, случайно "отключая" часть нейронов во время обучения

---

#### 4. Проекционный слой (`self.projection`)

```python
self.projection = nn.Linear(hidden_dim, vocab_size)
```

- **Что делает**: преобразует скрытое состояние сети в логиты — значения, из которых затем строится распределение вероятностей по всем символам словаря
- **Размерность выхода**: `[batch_size, seq_len, vocab_size]`
- **Итог**: для каждой позиции в последовательности мы получаем вероятности всех возможных следующих символов

---

#### 🔄 Прямой проход (`forward`)

Метод `forward` определяет, как данные проходят через модель:

1. **Эмбеддинги**: индексы превращаются в векторы
2. **GRU**: обработка последовательности с учётом контекста
3. **Дополнительная обработка**: линейное преобразование + Tanh + Dropout
4. **Проекция**: получаем логиты — ненормированные вероятности следующего символа

---

#### 💡 Почему именно такая архитектура?

Эта модель представляет собой **классический пример рекуррентной сети для языкового моделирования**. Вот почему она работает хорошо:

| Компонент      | Зачем он нужен |
|----------------|----------------|
| GRU            | хорошо работает с последовательностями, запоминает контекст |
| Эмбеддинги     | представляют символы в числовом виде |
| Dropout        | защищает от переобучения |
| Дополнительные слои | увеличивают выразительность модели |
| Размерность 256 | баланс между скоростью и качеством |

---

#### 📦 Что получаем в итоге?

Модель, которая:
- ✅ Может обучаться на больших объёмах текста
- ✅ Улавливает закономерности на уровне символов
- ✅ Генерирует текст, напоминающий обучающие данные
- ✅ Имеет около **296 тысяч параметров** — достаточно компактна, но при этом эффективна

---

#### 🧩 На заметку

Хотя современные модели (например, Transformer) значительно превосходят RNN по качеству, понимание работы рекуррентных сетей остаётся важным этапом в освоении NLP. GRU — отличная отправная точка для изучения последовательного моделирования.

</details>

### Main model

<details> 
    <summary><em><strong> Управляемый рекуррентный блок (GRU)</strong></em></summary>

## 1. Введение и мотивация

### 1.1 Контекст появления GRU: история создания как упрощения LSTM

Gated Recurrent Unit (GRU) — это разновидность рекуррентной нейронной сети, которая была представлена миру в 2014 году. GRU возникла в контексте активных исследований в области нейронного машинного перевода и обработки естественного языка, в период, когда LSTM (Long Short-Term Memory) уже зарекомендовали себя как эффективные модели для работы с последовательными данными.

**Хронология появления GRU:**

- **2013-2014**: в рамках развития архитектур нейронного машинного перевода исследователи из Монреаля начали экспериментировать с вариациями рекуррентных сетей.
  
- **Июнь 2014**: Кюнхён Чо и соавторы публикуют статью ["Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation"](https://arxiv.org/abs/1406.1078), в которой впервые представлена архитектура GRU.

- **Сентябрь 2014**: в статье ["Empirical Evaluation of Gated Recurrent Neural Networks on Sequence Modeling"](https://arxiv.org/abs/1412.3555) Юаша Чунг и др. проводят первое систематическое сравнение LSTM и GRU.

GRU возникла из потребности в более простой, но при этом не менее эффективной альтернативе LSTM. Исследователи искали архитектуру, которая могла бы:

- Обрабатывать долговременные зависимости в последовательностях
- Быть вычислительно более эффективной и проще для обучения
- Требовать меньше параметров при сохранении выразительной способности

Ключевым наблюдением было то, что не все компоненты сложной архитектуры LSTM необходимы для достижения хороших результатов. Это привело к созданию GRU, которая объединяет функциональность нескольких вентилей LSTM и упрощает общий механизм обновления состояния.

### 1.2 Авторы и ключевые публикации: Работы Чо и коллег, связь с машинным переводом

GRU неразрывно связана с именем **Кюнхёна Чо** (Kyunghyun Cho) и его коллегами из Монреальского университета. Исследовательская группа, работавшая над созданием GRU, включала таких учёных как Йошуа Бенджио (Yoshua Bengio), Дмитрий Бахданау (Dzmitry Bahdanau) и другие видные исследователи в области глубокого обучения.

**Основополагающие публикации:**

1. **"Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation" (Чо и др., 2014)**
   - Первое представление GRU в контексте архитектуры энкодер-декодер для машинного перевода
   - Демонстрация возможности обучения представлений фраз на уровне предложений
   - Сравнение с базовой RNN и показ преимуществ механизма вентилей

2. **"Empirical Evaluation of Gated Recurrent Neural Networks on Sequence Modeling" (Чунг и др., 2014)**
   - Систематическое сравнение LSTM и GRU на задачах моделирования полифонической музыки и обработки речевых сигналов
   - Вывод о сопоставимой производительности GRU с LSTM при меньшей вычислительной сложности

3. **"Neural Machine Translation by Jointly Learning to Align and Translate" (Бахданау, Чо и Бенджио, 2014)**
   - Применение GRU в архитектуре с механизмом внимания для машинного перевода
   - Демонстрация возможностей GRU в комбинации с механизмом внимания

**Связь с машинным переводом:**

Появление GRU тесно связано с развитием нейронного машинного перевода (NMT). В 2014 году исследователи активно искали альтернативы статистическим методам машинного перевода, и рекуррентные нейронные сети показывали многообещающие результаты.

GRU была разработана специально в контексте модели энкодер-декодер для NMT, где:
- **Энкодер** кодирует входное предложение в фиксированный вектор
- **Декодер** генерирует перевод на основе этого вектора

Работа Чо и соавторов показала, что GRU может эффективно кодировать семантическую и синтаксическую информацию предложений, что критически важно для качественного перевода. Более того, упрощенная структура GRU позволяла обучать более глубокие модели и масштабировать их на большие объемы данных, что было важно для практических приложений машинного перевода.

Цитата Кюнхёна Чо о создании GRU:
> "Мы стремились создать более простую альтернативу LSTM, которая могла бы быть легче в обучении и более эффективной вычислительно, сохраняя при этом способность моделировать долговременные зависимости в последовательных данных."

### 1.3 Баланс сложности и эффективности: Почему возникла потребность в более компактных моделях

В начале 2010-х годов LSTM стали стандартом де-факто для задач обработки последовательностей, но они имели несколько ограничений, которые стимулировали поиск более компактных альтернатив:

**Проблемы, связанные со сложностью LSTM:**

1. **Вычислительные требования**
   - LSTM имеет четыре набора весов и смещений (для вентилей забывания, входа, выхода и кандидат-вектора)
   - Обучение больших LSTM моделей требовало значительных вычислительных ресурсов
   - Время обработки одного элемента последовательности было критично для приложений реального времени

2. **Сложность обучения**
   - Большее количество параметров означало большее пространство поиска при оптимизации
   - LSTM часто требовали более тщательной настройки гиперпараметров
   - Сложнее добиться сходимости на ограниченных наборах данных

3. **Память и энергопотребление**
   - Модели с LSTM занимали больше памяти при развертывании
   - Это ограничивало их применение на мобильных и встраиваемых устройствах
   - Высокое энергопотребление при вычислениях

4. **Теоретическое понимание**
   - Сложная архитектура LSTM затрудняла анализ её поведения
   - Не всегда было ясно, какие компоненты вносят наибольший вклад в эффективность

**Почему потребовались более компактные модели:**

1. **Развитие мобильных и встраиваемых систем**
   - Рост потребности в моделях, способных работать на устройствах с ограниченными ресурсами
   - Необходимость баланса между точностью и энергоэффективностью

2. **Масштабирование к большим объемам данных**
   - Увеличение доступных наборов данных требовало более эффективных моделей
   - Более простые модели могли обрабатывать больше данных при тех же ресурсах

3. **Эксперименты по упрощению архитектуры**
   - Исследователи начали систематически изучать, какие компоненты LSTM действительно необходимы
   - Эксперименты показали, что некоторые элементы LSTM можно объединить без потери производительности

4. **Стремление к элегантности в дизайне**
   - Принцип Оккама: если две модели показывают одинаковую производительность, предпочтительнее более простая
   - Более простые модели часто лучше обобщаются на новые данные

**Как GRU решает эти проблемы:**

1. **Меньше параметров**
   - GRU имеет только два вентиля вместо трех в LSTM
   - Объединение функциональности вентилей входа и забывания в один вентиль обновления
   - Отсутствие отдельного состояния ячейки (используется только скрытое состояние)

2. **Сохранение ключевой функциональности**
   - Несмотря на упрощения, GRU сохраняет способность моделировать долговременные зависимости
   - Механизм вентилей всё ещё позволяет контролировать поток информации через сеть

3. **Эмпирические результаты**
   - На многих задачах GRU показала результаты, сопоставимые с LSTM
   - В некоторых случаях даже превосходила LSTM, особенно при ограниченных наборах данных

GRU представляет собой изящное балансирование между сложностью и эффективностью, которое позволило сделать рекуррентные модели более доступными и применимыми в широком спектре задач.

## 2. Архитектура GRU: ключевые компоненты

### 2.1 Интуиция: метафора "экономной памяти" с двумя контрольными точками

Для понимания принципа работы GRU давайте представим её как систему "экономной памяти" с двумя основными контрольными точками. Эта метафора поможет интуитивно понять, как GRU управляет информацией.

**Представьте библиотекаря, работающего с одной большой книгой (скрытое состояние):**

В отличие от LSTM, где есть отдельная книга для долговременного хранения (ячейка памяти) и рабочая тетрадь для текущих заметок (скрытое состояние), библиотекарь GRU работает только с одной книгой. В этой книге он постоянно обновляет информацию, следуя двум простым правилам:

**1. Вентиль сброса (Reset Gate) — "Что стоит перечитать?"**

Представьте, что библиотекарь решает, какие части его текущих знаний (хранящихся в книге) актуальны для понимания новой информации:

- Когда вентиль сброса близок к 1: "Эта часть моих текущих знаний важна для понимания новой информации"
- Когда вентиль сброса близок к 0: "Эта часть моих знаний не имеет отношения к новой информации, я её временно игнорирую"

Например, если вы читаете предложение "Погода в Париже...", и далее текст переключается на "...столице Франции", вентиль сброса может решить, что информация о "Париже" остаётся актуальной, а информация о "погоде" уже не важна для понимания продолжения предложения.

**2. Вентиль обновления (Update Gate) — "Насколько сильно обновить книгу?"**

После того, как библиотекарь определил, какая часть текущих знаний актуальна, он должен решить, в какой степени обновить содержимое книги:

- Когда вентиль обновления близок к 1: "Я полностью заменю эту часть книги новой информацией"
- Когда вентиль обновления близок к 0: "Я сохраню существующую информацию без изменений"

На примере текста: если до этого в книге была информация о Риме, а новое предложение говорит о Париже, вентиль обновления может установить высокое значение, чтобы заменить информацию о Риме на информацию о Париже.

**Процесс работы GRU в метафоре:**

1. **Получение новой информации**: библиотекарь получает новую порцию информации (входной вектор $x_t$)

2. **Определение релевантности старых знаний**: библиотекарь использует вентиль сброса, чтобы определить, какие части существующей книги (скрытого состояния $h_{t-1}$) важны для обработки новой информации

3. **Создание чернового обновления**: на основе релевантных частей старых знаний и новой информации библиотекарь создает черновик обновления (кандидат-вектор $\tilde{h}_t$)

4. **Решение о степени обновления**: используя вентиль обновления, библиотекарь решает, насколько сильно обновить каждую часть книги

5. **Обновление книги**: книга обновляется с учетом решений вентиля обновления, создавая новое состояние книги ($h_t$)

**Ключевое отличие от LSTM:**

LSTM можно сравнить с более сложной системой, где есть:
- Склад долговременного хранения (ячейка памяти)
- Рабочий стол (скрытое состояние)
- Три сотрудника, принимающих решения (три вентиля)

GRU упрощает эту систему, объединяя "склад" и "рабочий стол" в одно хранилище, и сокращая число "сотрудников" до двух, что делает систему более экономичной, сохраняя при этом её основную функциональность.

Эта экономичность — главное преимущество GRU, позволяющее достичь сопоставимых результатов с LSTM при меньших вычислительных затратах.

### 2.2 Формализация и обозначения: определение переменных и размерностей

Давайте формализуем архитектуру GRU, определив все её компоненты и их размерности. Это поможет как в понимании структуры, так и при последующей реализации.

**Основные обозначения:**

| **Символ** | **Размерность** | **Описание** |
|------------|-----------------|--------------|
| $x_t$ | $\mathbb{R}^{d_x}$ | Входной вектор на шаге $t$ |
| $h_t$ | $\mathbb{R}^{d_h}$ | Скрытое состояние на шаге $t$ |
| $z_t$ | $\mathbb{R}^{d_h}$ | Активация вентиля обновления (update gate) на шаге $t$ |
| $r_t$ | $\mathbb{R}^{d_h}$ | Активация вентиля сброса (reset gate) на шаге $t$ |
| $\tilde{h}_t$ | $\mathbb{R}^{d_h}$ | Кандидат-вектор нового скрытого состояния |

**Весовые матрицы и векторы смещения:**

| **Символ** | **Размерность** | **Описание** |
|------------|-----------------|--------------|
| $W_z$ | $\mathbb{R}^{d_h \times (d_x + d_h)}$ | Веса для вентиля обновления |
| $W_r$ | $\mathbb{R}^{d_h \times (d_x + d_h)}$ | Веса для вентиля сброса |
| $W_h$ | $\mathbb{R}^{d_h \times (d_x + d_h)}$ | Веса для кандидат-вектора |
| $b_z$ | $\mathbb{R}^{d_h}$ | Смещение для вентиля обновления |
| $b_r$ | $\mathbb{R}^{d_h}$ | Смещение для вентиля сброса |
| $b_h$ | $\mathbb{R}^{d_h}$ | Смещение для кандидат-вектора |

В альтернативной форме записи, можно разделить веса для входа и скрытого состояния:

| **Символ** | **Размерность** | **Описание** |
|------------|-----------------|--------------|
| $W_{xz}$ | $\mathbb{R}^{d_h \times d_x}$ | Веса для входа в вентиле обновления |
| $W_{hz}$ | $\mathbb{R}^{d_h \times d_h}$ | Веса для скрытого состояния в вентиле обновления |
| $W_{xr}$ | $\mathbb{R}^{d_h \times d_x}$ | Веса для входа в вентиле сброса |
| $W_{hr}$ | $\mathbb{R}^{d_h \times d_h}$ | Веса для скрытого состояния в вентиле сброса |
| $W_{xh}$ | $\mathbb{R}^{d_h \times d_x}$ | Веса для входа в кандидат-векторе |
| $W_{hh}$ | $\mathbb{R}^{d_h \times d_h}$ | Веса для скрытого состояния в кандидат-векторе |

Обе формы записи эквивалентны, но иногда удобнее использовать одну из них в зависимости от контекста.

**Функции активации:**

- $\sigma$: сигмоидальная функция, отображает входы в диапазон [0, 1]
- $\tanh$: гиперболический тангенс, отображает входы в диапазон [-1, 1]

**Размерности входных и выходных данных:**

- $d_x$: размерность входного вектора $x_t$
- $d_h$: размерность скрытого состояния

**Конкатенация входа и предыдущего состояния:**

Для упрощения записи мы часто используем конкатенацию входного вектора $x_t$ и предыдущего скрытого состояния $h_{t-1}$:

$$[x_t, h_{t-1}] \in \mathbb{R}^{d_x + d_h}$$

**Примечания по размерностям:**

1. В GRU используется только скрытое состояние $h_t$, в отличие от LSTM, где есть дополнительное состояние ячейки $C_t$.

2. Оба вентиля ($z_t$, $r_t$) имеют одинаковую размерность $d_h$, что позволяет им поэлементно контролировать обновление скрытого состояния.

3. Общее количество параметров в стандартном GRU:
   - Веса: $3 \times d_h \times (d_x + d_h)$
   - Смещения: $3 \times d_h$
   - Итого: $3 \times d_h \times (d_x + d_h + 1)$

Обратите внимание, что GRU имеет только 3/4 числа параметров LSTM, что является одним из ключевых преимуществ этой архитектуры.

### 2.3 Динамика одного шага

Теперь рассмотрим, как точно вычисляются все компоненты GRU на одном временном шаге $t$. Разберем каждый элемент архитектуры подробно.

![Image_01.png](https://raw.githubusercontent.com/Verbasik/Weekly-arXiv-ML-AI-Research-Review/refs/heads/develop/2025/week-17_&_18/assets/GRU/Image_01.jpg)

#### Вентиль сброса (reset gate)

Вентиль сброса $r_t$ определяет, какие элементы предыдущего состояния $h_{t-1}$ стоит учитывать при вычислении нового кандидат-вектора:

$$
r_t = \sigma\big(W_r \cdot [x_t, h_{t-1}] + b_r\big)
$$

или, в развернутой форме:

$$
r_t = \sigma\big(W_{xr} \cdot x_t + W_{hr} \cdot h_{t-1} + b_r\big)
$$

Здесь:
- $[x_t, h_{t-1}]$ — конкатенация текущего входа и предыдущего скрытого состояния
- $W_r$ (или $W_{xr}$ и $W_{hr}$) — весовые матрицы для вентиля сброса
- $b_r$ — вектор смещения
- $\sigma$ — сигмоидальная функция, возвращающая значения в интервале [0, 1]

Результат $r_t$ представляет собой вектор значений между 0 и 1, где:
- **1** означает "полностью учитывать эту информацию из предыдущего состояния"
- **0** означает "полностью игнорировать эту информацию из предыдущего состояния"

**Пример:** При обработке текста, если предложение меняет тему, вентиль сброса может установить низкие значения, чтобы "забыть" контекст предыдущей темы при формировании нового кандидат-вектора.

#### Вентиль обновления (update gate)

![Image_02.png](https://raw.githubusercontent.com/Verbasik/Weekly-arXiv-ML-AI-Research-Review/refs/heads/develop/2025/week-17_&_18/assets/GRU/Image_02.jpg)

Вентиль обновления $z_t$ определяет, насколько сильно каждый элемент скрытого состояния должен быть обновлен:

$$
z_t = \sigma\big(W_z \cdot [x_t, h_{t-1}] + b_z\big)
$$

или:

$$
z_t = \sigma\big(W_{xz} \cdot x_t + W_{hz} \cdot h_{t-1} + b_z\big)
$$

Как и в случае с вентилем сброса, результатом является вектор значений в интервале [0, 1], где:
- **1** означает "полностью заменить старую информацию новой"
- **0** означает "полностью сохранить старую информацию без изменений"

**Важное наблюдение:** вентиль обновления в GRU играет роль, аналогичную комбинации вентилей забывания и входа в LSTM. Высокое значение $z_t$ означает "забыть" старую информацию и "запомнить" новую.

#### Кандидат-вектор скрытого состояния

Кандидат-вектор $\tilde{h}_t$ представляет собой "предложение" для нового скрытого состояния, но с учетом только той части предыдущего состояния, которую определил вентиль сброса:

$$
\tilde{h}_t = \tanh\big(W_h \cdot [x_t, r_t \odot h_{t-1}] + b_h\big)
$$

или:

$$
\tilde{h}_t = \tanh\big(W_{xh} \cdot x_t + W_{hh} \cdot (r_t \odot h_{t-1}) + b_h\big)
$$

Здесь:
- $r_t \odot h_{t-1}$ — поэлементное умножение вентиля сброса на предыдущее скрытое состояние
- $\tanh$ — гиперболический тангенс, нормализующий значения в диапазон [-1, 1]

**Ключевой момент:** вентиль сброса применяется к $h_{t-1}$ перед его использованием в вычислении кандидат-вектора, а не к полному скрытому состоянию. Это позволяет сети "забыть" часть прошлого состояния при вычислении новой информации, но не обязательно при обновлении полного состояния.

#### Финальное обновление скрытого состояния

Наконец, вычисляем новое скрытое состояние $h_t$, как взвешенную комбинацию предыдущего состояния и кандидат-вектора, где вес определяется вентилем обновления:

$$
h_t = (1 - z_t) \odot h_{t-1} + z_t \odot \tilde{h}_t
$$

Здесь:
- $(1 - z_t) \odot h_{t-1}$ — часть старой информации, которую мы решили сохранить
- $z_t \odot \tilde{h}_t$ — часть новой информации, которую мы решили добавить

**Важное наблюдение:** эта формула эквивалентна интерполяции между старым состоянием $h_{t-1}$ и новым кандидат-состоянием $\tilde{h}_t$, где $z_t$ определяет точку интерполяции для каждого элемента вектора.

**Итог: полный набор формул GRU**

Для удобства, вот полный набор формул, описывающих один шаг GRU:

$$
\begin{align}
z_t &= \sigma(W_z \cdot [x_t, h_{t-1}] + b_z) \\
r_t &= \sigma(W_r \cdot [x_t, h_{t-1}] + b_r) \\
\tilde{h}_t &= \tanh(W_h \cdot [x_t, r_t \odot h_{t-1}] + b_h) \\
h_t &= (1 - z_t) \odot h_{t-1} + z_t \odot \tilde{h}_t
\end{align}
$$

Эти четыре уравнения полностью описывают динамику GRU ячейки на одном временном шаге.

## 3. Математическое сравнение GRU с LSTM

### 3.1 Упрощения в архитектуре: какие элементы LSTM были объединены или удалены

GRU можно рассматривать как упрощенную версию LSTM, где некоторые компоненты были объединены или полностью исключены. Давайте систематически рассмотрим эти упрощения.

**1. Объединение состояний: устранение отдельной ячейки памяти**

В LSTM существуют два вида состояний:
- Состояние ячейки (cell state) $C_t$ — долговременная память
- Скрытое состояние (hidden state) $h_t$ — краткосрочная память и выход

В GRU эти два состояния объединены в одно — скрытое состояние $h_t$, которое выполняет обе функции.

Математическое последствие:
- LSTM: $C_t = f_t \odot C_{t-1} + i_t \odot \tilde{C}_t$, затем $h_t = o_t \odot \tanh(C_t)$
- GRU: $h_t = (1 - z_t) \odot h_{t-1} + z_t \odot \tilde{h}_t$

**2. Слияние вентилей входа и забывания**

В LSTM вентиль забывания $f_t$ определяет, какую часть старой информации сохранить, а вентиль входа $i_t$ — какую часть новой информации добавить. Эти решения принимаются независимо.

В GRU вентиль обновления $z_t$ определяет одновременно, какую часть старой информации заменить новой. Это создает жесткую связь: если вы добавляете X% новой информации, то обязательно забываете X% старой.

Математическое последствие:
- LSTM: $C_t = f_t \odot C_{t-1} + i_t \odot \tilde{C}_t$ (где $f_t$ и $i_t$ независимы)
- GRU: $h_t = (1 - z_t) \odot h_{t-1} + z_t \odot \tilde{h}_t$ (где $z_t$ и $(1-z_t)$ комплементарны)

**3. Изменение применения вентиля сброса**

В LSTM вентиль выхода $o_t$ применяется после вычисления всех других компонентов, чтобы определить, какую часть информации из ячейки передать в выходное скрытое состояние.

В GRU вентиль сброса $r_t$ применяется перед вычислением кандидат-вектора, определяя, какую часть предыдущего состояния учитывать при создании нового состояния.

Математическое последствие:
- LSTM: $h_t = o_t \odot \tanh(C_t)$ (вентиль выхода контролирует финальный выход)
- GRU: $\tilde{h}_t = \tanh(W_h \cdot [x_t, r_t \odot h_{t-1}] + b_h)$ (вентиль сброса влияет на создание кандидат-вектора)

**4. Устранение выходного вентиля**

LSTM имеет отдельный выходной вентиль $o_t$, который контролирует, какая информация из ячейки памяти должна быть видна во внешнем скрытом состоянии.

GRU не имеет такого вентиля — всё скрытое состояние всегда полностью доступно.

**5. Сравнительная таблица компонентов**

| **Компонент LSTM** | **Аналог в GRU** | **Примечание** |
|--------------------|------------------|----------------|
| Состояние ячейки $C_t$ | Отсутствует (объединено с $h_t$) | В GRU единое состояние |
| Скрытое состояние $h_t$ | Скрытое состояние $h_t$ | Аналогично в обеих архитектурах |
| Вентиль забывания $f_t$ | Часть вентиля обновления $(1-z_t)$ | В GRU комплементарно вентилю входа |
| Вентиль входа $i_t$ | Вентиль обновления $z_t$ | В GRU комплементарно вентилю забывания |
| Вентиль выхода $o_t$ | Отсутствует | В GRU нет фильтрации выхода |
| Кандидат-вектор $\tilde{C}_t$ | Кандидат-вектор $\tilde{h}_t$ | Аналогично, но в GRU влияет вентиль сброса |
| - | Вентиль сброса $r_t$ | Уникален для GRU |

**6. Сравнение уравнений**

**LSTM**:
$$
f_t = \sigma(W_f \cdot [x_t, h_{t-1}] + b_f)
$$
$$
i_t = \sigma(W_i \cdot [x_t, h_{t-1}] + b_i)
$$
$$
o_t = \sigma(W_o \cdot [x_t, h_{t-1}] + b_o)
$$
$$
\tilde{C}_t = \tanh(W_C \cdot [x_t, h_{t-1}] + b_C)
$$
$$
C_t = f_t \odot C_{t-1} + i_t \odot \tilde{C}_t
$$
$$
h_t = o_t \odot \tanh(C_t)
$$

**GRU**:
$$
z_t = \sigma(W_z \cdot [x_t, h_{t-1}] + b_z)
$$
$$
r_t = \sigma(W_r \cdot [x_t, h_{t-1}] + b_r)
$$
$$
\tilde{h}_t = \tanh(W_h \cdot [x_t, r_t \odot h_{t-1}] + b_h)
$$
$$
h_t = (1 - z_t) \odot h_{t-1} + z_t \odot \tilde{h}_t
$$

**Ключевое наблюдение**: GRU имеет меньше уравнений и параметров, обеспечивая более компактную архитектуру, которая тем не менее сохраняет ключевую функциональность управления потоком информации.

### 3.2 Поток градиентов в GRU: анализ решения проблемы затухающего градиента

GRU, как и LSTM, успешно решает проблему затухающего градиента, но делает это немного иначе. Давайте проанализируем, как именно в GRU организован поток градиентов.

**Напомним проблему в ванильных RNN:**

В обычных RNN градиент затухает из-за многократного умножения на якобианы с собственными значениями меньше 1:

$$\frac{\partial h_t}{\partial h_{t-k}} = \prod_{i=t-k+1}^{t} \frac{\partial h_i}{\partial h_{i-1}} = \prod_{i=t-k+1}^{t} \text{diag}(\tanh'(a_i)) \cdot W_{hh}$$

**Анализ потока градиентов в GRU:**

В GRU скрытое состояние обновляется по формуле:

$$h_t = (1 - z_t) \odot h_{t-1} + z_t \odot \tilde{h}_t$$

Вычислим градиент $\frac{\partial h_t}{\partial h_{t-1}}$, используя цепное правило:

$$\frac{\partial h_t}{\partial h_{t-1}} = \frac{\partial}{\partial h_{t-1}} [(1 - z_t) \odot h_{t-1} + z_t \odot \tilde{h}_t]$$

Раскрывая это выражение:

$$\frac{\partial h_t}{\partial h_{t-1}} = \underbrace{(1 - z_t)}_{\text{прямой путь}} + \underbrace{\frac{\partial z_t}{\partial h_{t-1}} \odot (\tilde{h}_t - h_{t-1})}_{\text{через } z_t} + \underbrace{z_t \odot \frac{\partial \tilde{h}_t}{\partial h_{t-1}}}_{\text{через } \tilde{h}_t}$$

**Ключевое наблюдение 1: прямой путь градиента**

Первый член $(1 - z_t)$ представляет собой прямой путь градиента. Если $z_t$ близко к 0 (т.е. решение сохранить большую часть старой информации), то градиент может течь почти без изменений через этот путь.

Сравнение с LSTM:
- В LSTM прямой путь идет через состояние ячейки: $\frac{\partial C_t}{\partial C_{t-1}} = f_t$
- В GRU прямой путь идет через скрытое состояние: $(1 - z_t)$

**Ключевое наблюдение 2: адаптивное обновление**

GRU не просто предотвращает затухание градиентов — она делает это адаптивно. Сеть обучается устанавливать $z_t$ близким к 0 для тех элементов, где важно сохранить долговременные зависимости.

**Поток градиентов через несколько шагов:**

Для $k$ шагов назад, градиент можно выразить как:

$$\frac{\partial h_t}{\partial h_{t-k}} = \prod_{i=t-k+1}^{t} \frac{\partial h_i}{\partial h_{i-1}}$$

Если для каждого шага $i$ значения $(1 - z_i)$ близки к 1 для некоторых элементов, то градиент может течь через эти элементы без существенного затухания. Это создает "информационные магистрали" для обратного распространения.

**Вентиль сброса и градиенты:**

Вентиль сброса $r_t$ влияет на градиенты через второй путь:

$$\frac{\partial \tilde{h}_t}{\partial h_{t-1}} = \frac{\partial}{\partial h_{t-1}} \tanh(W_h \cdot [x_t, r_t \odot h_{t-1}] + b_h)$$

Это включает:
- Прямое влияние $r_t \odot \frac{\partial}{\partial h_{t-1}} \tanh(...)$
- Косвенное влияние через $\frac{\partial r_t}{\partial h_{t-1}}$

Вентиль сброса позволяет GRU "забывать" определенные компоненты предыдущего состояния при вычислении кандидат-вектора, но это не препятствует потоку градиентов через основной путь $(1 - z_t)$.

**Сравнение с LSTM:**

| **Аспект** | **LSTM** | **GRU** |
|------------|----------|---------|
| Основной путь градиента | Через состояние ячейки: $\frac{\partial C_t}{\partial C_{t-1}} = f_t$ | Через скрытое состояние: $(1 - z_t)$ |
| Контроль потока | Три независимых вентиля | Два вентиля, один из которых с комплементарным эффектом |
| Количество путей градиента | Множество путей через $C_t$ и $h_t$ | Меньше путей, но с прямой магистралью |

Таким образом, GRU решает проблему затухающего градиента через адаптивный механизм, который создает прямые пути для потока градиента, аналогично LSTM, но с использованием меньшего количества компонентов.

**3. Потребление памяти**

**LSTM**:
- Хранение состояний: $O(2d_h)$ для $h_t$ и $C_t$
- Хранение промежуточных результатов для обратного прохода: $O(4d_h T)$ для последовательности длины $T$

**GRU**:
- Хранение состояний: $O(d_h)$ только для $h_t$
- Хранение промежуточных результатов: $O(3d_h T)$

Экономия памяти GRU по сравнению с LSTM значительна при длинных последовательностях.

**4. Параллелизация и аппаратное ускорение**

**LSTM**:
- 4 матричных умножения могут быть распараллелены или объединены в одно большое умножение
- Более сложные зависимости между компонентами могут снизить эффективность конвейеризации

**GRU**:
- 3 матричных умножения также поддаются параллелизации
- Меньше зависимостей между компонентами, что потенциально лучше для конвейеризации
- Вычисление $r_t \odot h_{t-1}$ создает дополнительную зависимость

**Практический эффект**:
на современных GPU и специализированных аппаратных ускорителях (TPU, NPU) преимущество GRU в скорости часто составляет 20-30% по сравнению с LSTM при одинаковом размере скрытого состояния.

**5. Масштабирование к большим моделям**

При увеличении размерности скрытого состояния $d_h$ разница в вычислительной эффективности становится более значительной:

- Для $d_h = 1024$, разница в количестве параметров: ~1.7 миллиона
- Для $d_h = 2048$, разница уже превышает 6 миллионов параметров

Это особенно важно для глубоких моделей с несколькими слоями, где экономия умножается на количество слоев.

**Таблица сравнения эффективности для различных размерностей**

| **Размерность** | **Параметры LSTM** | **Параметры GRU** | **Экономия** | **Экономия (%)** |
|-----------------|--------------------|--------------------|--------------|-------------------|
| $d_h = 128$ | 0.22M | 0.16M | 0.06M | 25% |
| $d_h = 256$ | 0.57M | 0.43M | 0.14M | 25% |
| $d_h = 512$ | 1.67M | 1.25M | 0.42M | 25% |
| $d_h = 1024$ | 6.03M | 4.52M | 1.51M | 25% |
| $d_h = 2048$ | 23.14M | 17.35M | 5.79M | 25% |

Эта вычислительная эффективность делает GRU особенно привлекательным выбором для задач с ограниченными ресурсами и для моделей, которые работают в режиме реального времени.

## Вывод

Gated Recurrent Unit (GRU) представляет собой важное усовершенствование в архитектуре рекуррентных нейронных сетей, которое успешно балансирует между эффективностью и вычислительной сложностью. Будучи разработанной как упрощенная альтернатива LSTM в 2014 году, GRU сохранила ключевую способность моделировать долговременные зависимости в последовательностях, при этом значительно сократив количество параметров и вычислительных затрат.

Основные преимущества GRU заключаются в:
1. Упрощенной архитектуре с двумя вентилями вместо трех у LSTM
2. Объединении скрытого состояния и ячейки памяти в одно состояние
3. Сопоставимой производительности с LSTM при меньших вычислительных ресурсах
4. Лучшей масштабируемости для больших моделей и длинных последовательностей

Математический анализ показывает, что GRU эффективно решает проблему затухающего градиента благодаря адаптивному механизму управления потоком информации. При этом архитектура GRU демонстрирует значительную экономию в потреблении памяти и вычислительных ресурсов по сравнению с LSTM, особенно при увеличении размерности скрытого состояния.

Таким образом, GRU представляет собой оптимальный выбор для задач обработки последовательностей, где требуется баланс между эффективностью и вычислительной эффективностью, особенно в условиях ограниченных ресурсов или необходимости работы в реальном времени.

</details>

In [ ]:
class CharLM(nn.Module):
    """
    Description:
    ---------------
        Модель языковой модели на базе GRU для посимвольной генерации.

    Args:
    ---------------
        hidden_dim: Размерность скрытого состояния
        vocab_size: Размер словаря

    Examples:
    ---------------
        >>> model = CharLM(hidden_dim=256, vocab_size=1000)
        >>> input_tensor = torch.LongTensor([[1, 2, 3, 4]])
        >>> output = model(input_tensor)
        >>> output.shape
        torch.Size([1, 4, 1000])
    """
    def __init__(self, hidden_dim: int, vocab_size: int) -> None:
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.rnn = nn.GRU(hidden_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, hidden_dim)
        self.projection = nn.Linear(hidden_dim, vocab_size)

        self.non_lin = nn.Tanh()
        self.dropout = nn.Dropout(p=0.1)

    def forward(self, input_batch: torch.Tensor) -> torch.Tensor:
        """
        Description:
        ---------------
            Прямой проход модели.

        Args:
        ---------------
            input_batch: Тензор с индексами входных символов

        Returns:
        ---------------
            torch.Tensor: Предсказанные логиты для каждого символа
        """
        # Преобразование индексов в эмбеддинги
        embeddings = self.embedding(input_batch)  # [batch_size, seq_len, hidden_dim]
        
        # Пропуск через рекуррентный слой
        output, _ = self.rnn(embeddings)  # [batch_size, seq_len, hidden_dim]
        
        # Дополнительные слои
        output = self.dropout(
            self.linear(self.non_lin(output))
        )  # [batch_size, seq_len, hidden_dim]
        
        # Проекция на размер словаря
        projection = self.projection(
            self.non_lin(output)
        )  # [batch_size, seq_len, vocab_size]

        return projection

In [ ]:
# Инициализация и обучение модели на посимвольных данных
model = CharLM(hidden_dim=256, vocab_size=len(vocab)).to(device)

num_params = sum(p.numel() for p in model.parameters())
print(model)
print(f"Number of model parameters: {num_params:,}")

CharLM(
  (embedding): Embedding(91, 256)
  (rnn): GRU(256, 256, batch_first=True)
  (linear): Linear(in_features=256, out_features=256, bias=True)
  (projection): Linear(in_features=256, out_features=91, bias=True)
  (non_lin): Tanh()
  (dropout): Dropout(p=0.1, inplace=False)
)
Number of model parameters: 507,227


In [15]:
model, history = train(train_dataloader, eval_dataloader, model, 10)

epoch:  10%|█         | 1/10 [00:35<05:15, 35.02s/it]


Epoch 001 train_loss: 2.1936     val_loss 1.7596 train_perplexirty 10.3569 val_perplexirty 5.8121


epoch:  20%|██        | 2/10 [01:09<04:36, 34.51s/it]


Epoch 002 train_loss: 1.6882     val_loss 1.6169 train_perplexirty 5.4163 val_perplexirty 5.0392


epoch:  30%|███       | 3/10 [01:43<04:00, 34.33s/it]

...


Epoch 008 train_loss: 1.4678     val_loss 1.4623 train_perplexirty 4.3410 val_perplexirty 4.3171


epoch:  90%|█████████ | 9/10 [05:08<00:34, 34.15s/it]


Epoch 009 train_loss: 1.4568     val_loss 1.4540 train_perplexirty 4.2933 val_perplexirty 4.2817


epoch: 100%|██████████| 10/10 [05:42<00:00, 34.24s/it]


Epoch 010 train_loss: 1.4475     val_loss 1.4468 train_perplexirty 4.2539 val_perplexirty 4.2509
Best val perplexirty: 4.250897


Функция для генерации последовательности

In [ ]:
def generate_sequence(
    model: nn.Module, 
    dict_2ind: Dict[str, int], 
    ind2dict: Dict[int, str], 
    starting_seq: str, 
    max_seq_len: int = 256
) -> str:
    """
    Description:
    ---------------
        Генерирует текстовую последовательность, начиная с заданной.

    Args:
    ---------------
        model: Обученная языковая модель
        dict_2ind: Словарь для преобразования символов в индексы
        ind2dict: Словарь для преобразования индексов в символы
        starting_seq: Начальная последовательность
        max_seq_len: Максимальная длина генерируемой последовательности

    Returns:
    ---------------
        str: Сгенерированная последовательность

    Examples:
    ---------------
        >>> result = generate_sequence(model, char2ind, ind2char, "привет")
        >>> print(result)
        'привет мир...'
    """
    device = 'cpu'
    model = model.to(device)
    
    # Преобразование начальной последовательности в индексы
    input_ids = [dict_2ind['<bos>']] + [
        dict_2ind.get(char, dict_2ind['<unk>']) for char in starting_seq
    ]
    input_ids = torch.LongTensor(input_ids).to(device)

    model.eval()
    with torch.no_grad():
        for i in range(max_seq_len):
            # Получение распределения вероятностей для следующего символа
            next_char_distribution = model(input_ids.unsqueeze(0))
            next_char_logits = next_char_distribution[0, -1, :]
            next_char = next_char_logits.argmax()
            
            # Добавление предсказанного символа к входной последовательности
            input_ids = torch.cat([input_ids, next_char.unsqueeze(0)])

            # Завершение генерации при появлении токена конца последовательности
            if next_char.item() == dict_2ind['<eos>']:
                break

    # Преобразование индексов обратно в символы
    words = ' '.join([ind2dict[idx.item()] for idx in input_ids])

    return words

In [ ]:
# Генерация примера текста
generate_sequence(model, char2ind, ind2char, starting_seq='источник ')

'<bos>источник может быть производственные программирования и программирования программирования в программировании в программировании в программировании в программировании в программировании в программировании в программировании в программировании в программировании в пр'

### Токенизация по словам

In [ ]:
# Очистка памяти
import gc
torch.cuda.empty_cache()
gc.collect()

1086

In [ ]:
# Повторное извлечение предложений
sentences = []

for sentence in tqdm(df['body_text']):
    sentences.extend(
        [x.lower() for x in sent_tokenize(sentence, language='russian') if len(x) < 256]
    )
    
print("Количество предложений", len(sentences))

# Обработка и токенизация на уровне слов
words = Counter()

for sentence in tqdm(sentences):
    for word in nltk.word_tokenize(sentence):
            words[word] += 1
            
vocab = set(['<unk>', '<bos>', '<eos>', '<pad>'])
vocab_size = 40000

for elem in words.most_common(vocab_size):
    vocab.add(elem[0])
    
print("Всего слов в словаре:", len(vocab))

100%|██████████| 3300/3300 [00:10<00:00, 319.10it/s]


Количество предложений 120873


100%|██████████| 120873/120873 [00:29<00:00, 4159.08it/s]


Всего слов в словаре: 40004


In [22]:
word2ind = {char: i for i, char in enumerate(vocab)}
ind2word = {i: char for char, i in word2ind.items()}

In [ ]:
class WordDataset:
    """
    Description:
    ---------------
        Датасет для работы с пословной токенизацией текста.

    Args:
    ---------------
        sentences: Список предложений для обработки

    Examples:
    ---------------
        >>> dataset = WordDataset(sentences)
        >>> sample = dataset[0]
        >>> len(sample)
        # Длина токенизированного предложения
    """
    def __init__(self, sentences: List[str]) -> None:
        self.data = sentences
        self.unk_id = word2ind['<unk>']
        self.bos_id = word2ind['<bos>']
        self.eos_id = word2ind['<eos>']
        self.pad_id = word2ind['<pad>']

    def __getitem__(self, idx: int) -> List[int]:
        """
        Description:
        ---------------
            Преобразует предложение в последовательность индексов слов.

        Args:
        ---------------
            idx: Индекс предложения в датасете

        Returns:
        ---------------
            List[int]: Токенизированное предложение как список индексов
        """
        tokenized_sentence = [self.bos_id]
        tokenized_sentence += [
            word2ind.get(word, self.unk_id) 
            for word in nltk.word_tokenize(self.data[idx])
        ]
        tokenized_sentence += [self.eos_id]
        
        return tokenized_sentence

    def __len__(self) -> int:
        """
        Description:
        ---------------
            Возвращает размер датасета.

        Returns:
        ---------------
            int: Количество предложений в датасете
        """
        return len(self.data)

In [ ]:
def collate_fn_with_padding(
    input_batch: List[List[int]], 
    pad_id: int = word2ind['<pad>']
) -> Dict[str, torch.Tensor]:
    """
    Description:
    ---------------
        Функция для преобразования батча данных: добавляет паддинг и 
        создает тензоры ввода и целевых значений.

    Args:
    ---------------
        input_batch: Пакет токенизированных предложений
        pad_id: Идентификатор токена для заполнения (паддинга)

    Returns:
    ---------------
        Dict[str, torch.Tensor]: Словарь с тензорами входных и целевых id
    """
    seq_lens = [len(x) for x in input_batch]
    max_seq_len = max(seq_lens)

    new_batch = []
    for sequence in input_batch:
        for _ in range(max_seq_len - len(sequence)):
            sequence.append(pad_id)
        new_batch.append(sequence)

    sequences = torch.LongTensor(new_batch).to(device)

    new_batch = {
        'input_ids': sequences[:, :-1],
        'target_ids': sequences[:, 1:]
    }

    return new_batch

In [ ]:
# Разделение данных на обучающую и тестовую выборки
train_sentences, eval_sentences = train_test_split(
    sentences, test_size=0.2
)

# Создание датасетов и загрузчиков данных для словесной модели
train_dataset = WordDataset(train_sentences)
eval_dataset = WordDataset(eval_sentences)

train_dataloader = DataLoader(
    train_dataset, 
    collate_fn=collate_fn_with_padding, 
    batch_size=64
)

eval_dataloader = DataLoader(
    eval_dataset, 
    collate_fn=collate_fn_with_padding, 
    batch_size=64
)

In [ ]:
# Инициализация и обучение модели на словесных данных
model = CharLM(hidden_dim=256, vocab_size=len(vocab)).to(device)

num_params = sum(p.numel() for p in model.parameters())
print(model)
print(f"Number of model parameters: {num_params:,}")

CharLM(
  (embedding): Embedding(40004, 256)
  (rnn): GRU(256, 256, batch_first=True)
  (linear): Linear(in_features=256, out_features=256, bias=True)
  (projection): Linear(in_features=256, out_features=40004, bias=True)
  (non_lin): Tanh()
  (dropout): Dropout(p=0.1, inplace=False)
)
Number of model parameters: 20,982,596


In [ ]:
best_model, losses = train(
    train_dataloader, 
    eval_dataloader, 
    model, 
    10, 
    ignore_index=word2ind["<pad>"]
)

epoch:  10%|█         | 1/10 [01:45<15:45, 105.00s/it]


Epoch 001 train_loss: 4.8116     val_loss 5.2675 train_perplexirty 125.5927 val_perplexirty 198.0715


epoch:  20%|██        | 2/10 [03:30<14:00, 105.06s/it]


Epoch 002 train_loss: 4.4128     val_loss 5.2255 train_perplexirty 83.7987 val_perplexirty 190.3353


epoch:  30%|███       | 3/10 [05:15<12:16, 105.16s/it]

...


Epoch 008 train_loss: 3.2282     val_loss 5.5834 train_perplexirty 25.5596 val_perplexirty 275.9186


epoch:  90%|█████████ | 9/10 [15:45<01:44, 104.99s/it]


Epoch 009 train_loss: 3.1171     val_loss 5.6592 train_perplexirty 22.8660 val_perplexirty 298.2022


epoch: 100%|██████████| 10/10 [17:30<00:00, 105.02s/it]


Epoch 010 train_loss: 3.0210     val_loss 5.7391 train_perplexirty 20.7655 val_perplexirty 323.7256
Best val perplexirty: 190.335309


In [ ]:
# Генерация текста с использованием словесной модели
generate_sequence(
    model, 
    word2ind, 
    ind2word, 
    starting_seq=nltk.word_tokenize('кот')
)

'<bos> кот высотой 2 марта 2008 года выборы президента российской федерации выборы уважаемый избиратель ! <eos>'

### Выводы

**Simple RNN**

GRU продемонстрировала базовую способность к генерации текста, GRU не имеет long памяти, как у LSTM (long short-term memory), поэтому на этапе создании модели было ясно, что качество генерации на больших последовательностях будет храмать очень сильно, а на маленьких последовательностях показывать приемлемые результаты. При посимвольной токенизации модель показала, как было сказано ранее, приемлимые результаты при генерации небольшого текста (2-4 слова). По-словная токенизация улучшила семантическую целостность текста, но остались такие же проблемы как и при посимвольной токенизации.

**Плюсы:**
- Простота реализации и быстрая скорость обучения на коротких текстах.
- Меньшая вычислительная сложность по сравнению с более сложными архитектурами.

**Минусы:**
- Плохая работа с длинными зависимостями.
- Посимвольная токенизация может приводить к потере смысла текста.